# Nucleus SDK Tutorial

This notebook demonstrates how to use the Nucleus SDK for various common tasks. We'll cover both basic and advanced usage patterns.

## Setup

First, let's install the SDK and import the necessary components:

In [ ]:
!pip install nucleus-sdk

In [ ]:
from nucleus import NucleusClient
from nucleus.async_client import AsyncNucleusClient
from nucleus.models import Severity, AssetType
from nucleus.exceptions import NucleusAPIError
import asyncio
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)

## Basic Usage

Let's start with basic operations using the synchronous client.

In [ ]:
# Initialize the client
client = NucleusClient(api_key="your-api-key-here")

# Get list of projects
try:
    projects = client.get_projects()
    print("Projects:")
    for project in projects:
        print(f"- {project.project_name} (ID: {project.project_id})")
except NucleusAPIError as e:
    print(f"Error: {e}")

### Working with Assets

Let's see how to manage assets in a project:

In [ ]:
def demonstrate_asset_operations(project_id):
    # Create a new asset
    new_asset = client.create_asset(
        project_id=project_id,
        asset_name=f"demo-server-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
        asset_type=AssetType.HOST,
        ip_address="192.168.1.100",
        operating_system_name="Ubuntu",
        operating_system_version="20.04",
        asset_groups=["demo-group"]
    )
    print(f"Created new asset: {new_asset}")
    
    # Get asset details
    asset = client.get_asset(project_id, new_asset['asset_id'])
    print(f"\nAsset details: {asset}")
    
    # Update asset
    client.update_asset(
        project_id=project_id,
        asset_id=new_asset['asset_id'],
        asset_notes="Updated via SDK tutorial"
    )
    print("\nAsset updated successfully")

if projects:
    demonstrate_asset_operations(projects[0].project_id)

### Working with Findings

Now let's look at how to work with findings:

In [ ]:
def demonstrate_findings_operations(project_id):
    # Search for critical findings
    findings = client.search_findings(
        project_id=project_id,
        filters=[{
            "property": "finding_severity",
            "value": Severity.CRITICAL,
            "exact_match": True
        }]
    )
    
    print("Critical findings:")
    for finding in findings:
        print(f"- {finding.finding_name}")
        print(f"  Severity: {finding.finding_severity}")
        print(f"  Status: {finding.finding_status}")
        print(f"  Discovered: {finding.finding_discovered}")
        print()

if projects:
    demonstrate_findings_operations(projects[0].project_id)

## Advanced Usage

Now let's explore some advanced features using the async client.

In [ ]:
async def demonstrate_async_operations():
    async with AsyncNucleusClient(
        api_key="your-api-key-here",
        cache_ttl=300,  # 5 minutes cache
        rate_limit_calls=100,  # 100 calls per minute
        rate_limit_period=60
    ) as client:
        # Fetch multiple resources concurrently
        print("Fetching data concurrently...")
        start_time = datetime.now()
        
        projects, findings = await asyncio.gather(
            client.get_projects(),
            client.search_findings(
                project_id=123,
                filters=[{
                    "property": "finding_severity",
                    "value": Severity.CRITICAL,
                    "exact_match": True
                }]
            )
        )

        end_time = datetime.now()
        print(f"Concurrent fetching completed in {(end_time - start_time).total_seconds():.2f} seconds")
        
        return projects, findings

# Run async example
projects, findings = await demonstrate_async_operations()

### Bulk Operations

Let's see how to perform bulk operations efficiently:

In [ ]:
async def demonstrate_bulk_operations(project_id):
    async with AsyncNucleusClient(api_key="your-api-key-here") as client:
        # Prepare bulk updates
        updates = [
            {
                "finding_number": "VULN-001",
                "finding_status": "In Progress",
                "comment": "Working on fix"
            },
            {
                "finding_number": "VULN-002",
                "finding_status": "In Progress",
                "comment": "Under review"
            }
        ]
        
        try:
            result = await client.bulk_update_findings(project_id, updates)
            print(f"Bulk update completed: {result}")
        except NucleusAPIError as e:
            print(f"Bulk update failed: {e}")

if projects:
    await demonstrate_bulk_operations(projects[0].project_id)

### Parallel Asset Scanning

Here's how to scan multiple assets in parallel:

In [ ]:
async def scan_assets_parallel(project_id):
    async with AsyncNucleusClient(api_key="your-api-key-here") as client:
        # Get all assets
        assets = await client.get_project_assets(project_id)
        
        # Get findings for each asset in parallel
        print(f"\nFetching findings for {len(assets)} assets in parallel...")
        start_time = datetime.now()
        
        asset_findings = await asyncio.gather(
            *[client.get_asset_findings(project_id, asset.asset_id) for asset in assets[:5]],
            return_exceptions=True
        )
        
        end_time = datetime.now()
        print(f"Completed in {(end_time - start_time).total_seconds():.2f} seconds")
        
        # Process results
        for asset, findings in zip(assets[:5], asset_findings):
            if isinstance(findings, Exception):
                print(f"Error for asset {asset.asset_name}: {findings}")
            else:
                print(f"\nFindings for {asset.asset_name}:")
                for finding in findings[:3]:  # Show first 3 findings
                    print(f"- {finding.finding_name} ({finding.finding_severity})")

if projects:
    await scan_assets_parallel(projects[0].project_id)

## Error Handling Examples

Let's look at proper error handling:

In [ ]:
def demonstrate_error_handling():
    try:
        # Try to get a non-existent project
        project = client.get_project(project_id=999999)
    except NucleusNotFoundError:
        print("Project not found")
    except NucleusAuthError:
        print("Authentication failed")
    except NucleusPermissionError:
        print("Permission denied")
    except NucleusAPIError as e:
        print(f"API error: {e}")
        if hasattr(e, 'status_code'):
            print(f"Status code: {e.status_code}")
        if hasattr(e, 'response'):
            print(f"Response: {e.response}")

demonstrate_error_handling()

## Conclusion

This notebook demonstrated various features of the Nucleus SDK, including:
- Basic operations with projects, assets, and findings
- Async operations for improved performance
- Bulk operations for efficient updates
- Parallel asset scanning
- Proper error handling

For more information, check out the [SDK documentation](https://github.com/your-repo/nucleus-sdk) and the [Nucleus API Documentation](https://api-docs.nucleussec.com/nucleus/docs/).